In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np
import pandas as pd
from time import sleep
from random import randint
import datetime

In [2]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
city = 'Chicago'

# Create empty list to store scraped data
datatoget = []

for counter, i in enumerate(range(0,960,120)):   
    
    # Set url
    url = f'https://chicago.craigslist.org/search/apa?s={i}'
    browser.visit(url)
    sleep(randint(5,7))

    # Parse the HTML
    html = browser.html
    soup = bs(html, 'html.parser')

    # Find and extract relevant data
    search = soup.find_all('div', class_='result-info')

    print('\n====================================================')
    print(f'This page ({counter+1}) displays a range of {i} to {i+120} entries')
    print('====================================================')    


    # Loop through data found and extract relevant information
    for (counter,link) in enumerate(search):
        
        # Scrape dates
        dates = link.find_all('time', class_='result-date')[0].text.split('·')[0]

        # Scrape titles
        titles = link.find_all('a', class_='result-title hdrlnk')[0].text

        # Scrape links
        links = link.find_all('a', class_='result-title hdrlnk', href=True)[0]['href']

        try:
        # Scrape prices
            prices = link.find_all('span', class_='result-price')[0].text
        except:
            prices = np.nan

        # Scrape neighborhoods
        try:
            neighborhoods = link.find('span', class_='result-hood').text.strip()[1:-1]

        except:
            neighborhoods = np.nan

        # Visit each link seperately
        browser.visit(links)

        # Ensures that the server does not get overloaded with requests
        sleep(randint(2,5))  

        # After visiting link, extract bath and amenities data
        html = browser.html
        soup = bs(html, 'html.parser')
        
        try:
            baths = soup.find_all('p', class_='attrgroup')[0].text.split('\n')[1].split()[2][:-2]

        except:
            baths = np.nan

        try:
            if soup.find_all('p', class_='attrgroup')[0].text.split('\n')[2][-3:] == 'ft2':
                sqft = soup.find_all('p', class_='attrgroup')[0].text.split('\n')[2][:-3]
            else:
                sqft = np.nan
        except:
            sqft = np.nan    


        # Error handeling in case there are no bedrooms displayed
        try:
            bedrooms = soup.find_all('p', class_='attrgroup')[0].text.split('\n')[1].split('/')[0].strip().lower()

        except:
            bedrooms = np.nan

        amenities = soup.find_all('p', class_='attrgroup')[1].text.split('\n')
        # List comprehension used to make sure no empty strings get added to the list
        all_amenities = [am for am in amenities if am != '']

        # Print statements to ensure that the correct data is scraped
        print(f'\nListing Number: {counter+1}')
        print(f'Square Footage: {sqft if sqft == sqft else "not available"}')
        print(f'Price: {prices if prices == prices else "not available"}')
        print(f'Title: {titles}')
        print(f'Date: {dates}')
        print(f'Number of Bedrooms: {bedrooms if bedrooms == bedrooms else "not available"}')
        print(f'Neighborhood: {neighborhoods if neighborhoods == neighborhoods else "not available"}')
        print(f'Number of Bathrooms: {baths if baths == baths else "not available"}')
        print(f'Link: {links}')
        print(f'Amenities: {all_amenities}')

        # Create dicitonary for our DataFrame
        data = {'date': dates,
                'title': titles,
                'link': links,
                'price': prices,
                'bedroom': bedrooms,
                'sqft': sqft,
                'neighborhood': neighborhoods,
                'bathroom': baths,
                'amenities': all_amenities}

        # Append the data
        datatoget.append(data)

print('\n================ Done ================')

# Quit automated browser instance
browser.quit()


This page (1) displays a range of 0 to 120 entries

Listing Number: 1
Square Footage: not available
Price: $1,670
Title: Beautiful studio with washer dryer in unit &bldg amenities
Date: Jan 17
Number of Bedrooms: 0br
Neighborhood: Gold Coast/River North
Number of Bathrooms: 1
Link: https://chicago.craigslist.org/chc/apa/d/chicago-beautiful-studio-with-washer/7579411893.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'attached garage', 'rent period: monthly']

Listing Number: 2
Square Footage: not available
Price: $1,675
Title: Awesome 1Bed with ton of building amenities like gorgeous outdoor pool
Date: Jan 17
Number of Bedrooms: 1br
Neighborhood: Gold Coast
Number of Bathrooms: 1
Link: https://chicago.craigslist.org/chc/apa/d/chicago-awesome-1bed-with-ton-of/7579411682.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'laundry in bldg', 'attached garage', 'rent period: monthly']

Listing Number: 3
Square Footage: not


Listing Number: 19
Square Footage: not available
Price: $2,183
Title: Beautiful 1 Bedroom in The Loop - Unbeatable Location Downtown w/ Pool
Date: Jan 16
Number of Bedrooms: 1br
Neighborhood: The Loop
Number of Bathrooms: 1
Link: https://chicago.craigslist.org/chc/apa/d/chicago-beautiful-bedroom-in-the-loop/7579371783.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'attached garage', 'rent period: monthly']

Listing Number: 20
Square Footage: 450
Price: $1,250
Title: 5501 S. Cornell 02
Date: Jan 16
Number of Bedrooms: 0br
Neighborhood: Chicago
Number of Bathrooms: 1
Link: https://chicago.craigslist.org/chc/apa/d/chicago-5501-cornell-02/7579370687.html
Amenities: ['apartment', 'laundry on site', 'no parking']

Listing Number: 21
Square Footage: 400
Price: $1,250
Title: 5501 S. Cornell 15
Date: Jan 16
Number of Bedrooms: 0br
Neighborhood: Chicago
Number of Bathrooms: 1
Link: https://chicago.craigslist.org/chc/apa/d/chicago-5501-cornell-15/75793


Listing Number: 39
Square Footage: 700
Price: $2,050
Title: Valuable 1 Bedroom, 1 Bath in Lincoln Park, Available: 2/1/2023
Date: Jan 16
Number of Bedrooms: 1br
Neighborhood: Lincoln Park
Number of Bathrooms: 1
Link: https://chicago.craigslist.org/chc/apa/d/chicago-valuable-bedroom-bath-in/7579344423.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'street parking', 'rent period: monthly']

Listing Number: 40
Square Footage: 750
Price: $1,750
Title: Lakeview Apartment for Relet - I pay your first month's rent
Date: Jan 16
Number of Bedrooms: 1br
Neighborhood: Lakeview/Northalsted
Number of Bathrooms: 1
Link: https://chicago.craigslist.org/chc/apa/d/chicago-lakeview-apartment-for-relet/7579344403.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'laundry in bldg', 'street parking', 'rent period: monthly']

Listing Number: 41
Square Footage: 700
Price: $1,950
Title: Mammoth 1 Bedroom, 1 Bath in Gold Coast, Available: 2/


Listing Number: 58
Square Footage: not available
Price: $1,828
Title: Heart Of Ravenswood & One Month Free Schedule Today
Date: Jan 16
Number of Bedrooms: 1br
Neighborhood: Ravenswood
Number of Bathrooms: 1
Link: https://chicago.craigslist.org/chc/apa/d/chicago-heart-of-ravenswood-one-month/7579333512.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'street parking', 'rent period: monthly']

Listing Number: 59
Square Footage: 700
Price: $1,795
Title: Praiseworthy 1 Bedroom, 1 Bath in Wrigleyville, Available: 2/1/2023
Date: Jan 16
Number of Bedrooms: 1br
Neighborhood: Wrigleyville
Number of Bathrooms: 1
Link: https://chicago.craigslist.org/chc/apa/d/chicago-praiseworthy-bedroom-bath-in/7579332940.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'street parking', 'rent period: monthly']

Listing Number: 60
Square Footage: not available
Price: $2,110
Title: 2 Bed Available in Hot South Loop- Close to Publ


Listing Number: 77
Square Footage: 850
Price: $1,350
Title: Sunny East Pilsen Apartment
Date: Jan 16
Number of Bedrooms: 2br
Neighborhood: East Pilsen
Number of Bathrooms: 1
Link: https://chicago.craigslist.org/chc/apa/d/chicago-sunny-east-pilsen-apartment/7579324662.html
Amenities: ['apartment', 'no laundry on site', 'street parking', 'rent period: monthly']

Listing Number: 78
Square Footage: 700
Price: $1,425
Title: Priceless 1 Bedroom, 1 Bath in Ravenswood, Available: Now
Date: Jan 16
Number of Bedrooms: 1br
Neighborhood: Ravenswood
Number of Bathrooms: 1
Link: https://chicago.craigslist.org/chc/apa/d/chicago-priceless-bedroom-bath-in/7579324305.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'laundry in bldg', 'street parking', 'rent period: monthly']

Listing Number: 79
Square Footage: 800
Price: $900
Title: APARTMENT FOR RENT
Date: Jan 16
Number of Bedrooms: 1br
Neighborhood: EVERGREEN PARK, IL
Number of Bathrooms: 1
Link: https://chicago.craigslist.

...


Listing Number: 30
Square Footage: not available
Price: $2,000
Title: Amazing 1 bed in River North with w/d in unit & rooftop deck w pool
Date: Jan 12
Number of Bedrooms: 1br
Neighborhood: River North
Number of Bathrooms: 1
Link: https://chicago.craigslist.org/chc/apa/d/chicago-amazing-bed-in-river-north-with/7577627265.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'attached garage', 'rent period: monthly']

Listing Number: 31
Square Footage: not available
Price: $2,950
Title: 2Bed with w/d in unit, hw floors, ss appliances, gym, garden doorstaff
Date: Jan 12
Number of Bedrooms: 2br
Neighborhood: Fulton/West Loop
Number of Bathrooms: 2
Link: https://chicago.craigslist.org/chc/apa/d/chicago-2bed-with-d-in-unit-hw-floors/7577627097.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'attached garage', 'rent period: monthly']

Listing Number: 32
Square Footage: not available
Price: $2,150
Title: 1bed in W


Listing Number: 49
Square Footage: 850
Price: $1,620
Title: 2br 1ba Irving Park semi furnished
Date: Jan 11
Number of Bedrooms: 2br
Neighborhood: Chicago
Number of Bathrooms: 1
Link: https://chicago.craigslist.org/chc/apa/d/chicago-2br-1ba-irving-park-semi/7577563570.html
Amenities: ['air conditioning', 'dogs are OK - wooof', 'furnished', 'apartment', 'w/d in unit', 'street parking', 'rent period: monthly']

Listing Number: 50
Square Footage: 325
Price: $1,150
Title: Cool, Crisp Modern Design Studio in East RoPa—Available Now!
Date: Jan 11
Number of Bedrooms: 0br
Neighborhood: Rogers Park
Number of Bathrooms: 1
Link: https://chicago.craigslist.org/chc/apa/d/chicago-cool-crisp-modern-design-studio/7577559850.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'laundry in bldg', 'no smoking', 'street parking', 'rent period: monthly']

Listing Number: 51
Square Footage: not available
Price: $2,200
Title: Wicker Park - Division and Ashland
Date: Jan 11
Number of Be


Listing Number: 68
Square Footage: 550
Price: $1,403
Title: $1,403 (everything included) 1 bedroom luxury apartment uptown sublet
Date: Jan 11
Number of Bedrooms: 1br
Neighborhood: Uptown Chicago
Number of Bathrooms: 1
Link: https://chicago.craigslist.org/chc/apa/d/chicago-1403-everything-included/7577503774.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'laundry in bldg', 'detached garage', 'rent period: monthly', 'wheelchair accessible']

Listing Number: 69
Square Footage: 1128
Price: $2,095
Title: You've never seen anything like these deals! Call now!
Date: Jan 11
Number of Bedrooms: 3br
Neighborhood: Arlington Heights
Number of Bathrooms: 2
Link: https://chicago.craigslist.org/nwc/apa/d/arlington-heights-youve-never-seen/7577503086.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'laundry on site', 'street parking', 'rent period: monthly']

Listing Number: 70
Square Footage: not available
Price: $1,745
Tit


Listing Number: 86
Square Footage: not available
Price: $1,620
Title: Streeterville Jr 1 bedroom, lg closets, close to lake, low utilities
Date: Jan 11
Number of Bedrooms: 1br
Neighborhood: Streeterville
Number of Bathrooms: 1
Link: https://chicago.craigslist.org/chc/apa/d/chicago-streeterville-jr-bedroom-lg/7577453473.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'laundry on site', 'attached garage', 'rent period: monthly']

Listing Number: 87
Square Footage: 900
Price: $1,950
Title: REHAB 2 Bedroom, Ravenswood, InUnit Laundry, Pets ok, Close to EL
Date: Jan 11
Number of Bedrooms: 2br
Neighborhood: Ravenswood
Number of Bathrooms: 1
Link: https://chicago.craigslist.org/chc/apa/d/chicago-rehab-bedroom-ravenswood-inunit/7577450124.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'street parking', 'rent period: monthly']

Listing Number: 88
Square Footage: 1600
Price: $2,775
Title: All new kitchen with stainless appl


Listing Number: 105
Square Footage: 878
Price: $1,475
Title: This is the place you've been looking for! 2 BD / 1 BA!
Date: Jan 11
Number of Bedrooms: 2br
Neighborhood: Lombard
Number of Bathrooms: 1
Link: https://chicago.craigslist.org/nwc/apa/d/glen-ellyn-this-is-the-place-youve-been/7577407188.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'off-street parking', 'rent period: monthly', 'wheelchair accessible']

Listing Number: 106
Square Footage: not available
Price: $2,929
Title: Large South Loop 2 Bedroom. Big Windows, W/D,2 Balcony! Condo Quality
Date: Jan 11
Number of Bedrooms: 2br
Neighborhood: South Loop
Number of Bathrooms: 2
Link: https://chicago.craigslist.org/chc/apa/d/chicago-large-south-loop-bedroom-big/7577403091.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'attached garage', 'rent period: monthly']

Listing Number: 107
Square Footage: 850
Price: $1,100
Title: Small 2 bedroom with h

In [4]:
# creating DataFrame
df = pd.DataFrame(datatoget)
df

,date,title,link,price,bedroom,sqft,neighborhood,bathroom,amenities
0,Jan 17,Beautiful studio with washer dryer in unit &bl...,https://chicago.craigslist.org/chc/apa/d/chica...,"$1,670",0br,NaN,Gold Coast/River North,1,"[cats are OK - purrr, dogs are OK - wooof, apa..."
1,Jan 17,Awesome 1Bed with ton of building amenities li...,https://chicago.craigslist.org/chc/apa/d/chica...,"$1,675",1br,NaN,Gold Coast,1,"[cats are OK - purrr, dogs are OK - wooof, apa..."
2,Jan 17,Fabulous unit in the heart of West Loop with w...,https://chicago.craigslist.org/chc/apa/d/chica...,"$1,850",1br,NaN,West Loop,1,"[cats are OK - purrr, dogs are OK - wooof, apa..."
3,Jan 17,"Gorgeous studio with w/d in unit, 24hr doorman...",https://chicago.craigslist.org/chc/apa/d/chica...,"$1,600",0br,NaN,Fulton Market/River west,1,"[cats are OK - purrr, dogs are OK - wooof, apa..."
4,Jan 17,"Pet friendly 1 bed with w/d, hw floors, and lo...",https://chicago.craigslist.org/chc/apa/d/chica...,"$1,750",1br,NaN,Loop,1,"[cats are OK - purrr, dogs are OK - wooof, apa..."
...,...,...,...,...,...,...,...,...,...
955,Jan 11,LOVELY AND COZY 2 BEDROOM & OFFICE,https://chicago.craigslist.org/wcl/apa/d/frank...,"$2,500",2br,1300,25TH/ SCHILLER BLVD IN FRANKLIN PARK,1,"[cats are OK - purrr, dogs are OK - wooof, apa..."
956,Jan 11,"Luxury Studio w/Amenities, Huge Closet,W/D, Gy...",https://chicago.craigslist.org/chc/apa/d/chica...,"$2,131",0br,NaN,West Loop,1,"[cats are OK - purrr, dogs are OK - wooof, apa..."
957,Jan 11,Gold Coast 1 BR 21st Floor w/N & E Views!,https://chicago.craigslist.org/chc/apa/d/chica...,"$1,500",1br,750,Sandburg Village,1,"[air conditioning, application fee details: $5..."
958,Jan 11,North Center Apartment for Rent,https://chicago.craigslist.org/chc/apa/d/chica...,"$1,250",1br,500,4117 N. Wolcott #2,1,"[cats are OK - purrr, dogs are OK - wooof, apa..."


In [5]:
# Exporting data to csv
df.to_csv(f"{city}_data_raw.csv", index=False)